In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 21723647
paper_name = 'bendaha_bellaoui_2011' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
files = ['hap_compound3.txt','het_compound3.txt', 'hom_compound6.txt','het_compound6.txt']

In [7]:
original_data_list = []
for f in files:
    original_data = pd.read_csv('raw_data/' + f, header=None, names=['genes'])
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    original_data['genes'] = original_data['genes'].astype(str)
    # Eliminate all white spaces & capitalize
    original_data['genes'] = clean_genename(original_data['genes'])
    # Translate to ORFs 
    original_data['orf'] = translate_sc(original_data['genes'], to='orf')
    # Make sure everything translated ok
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data['data'] = -1
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 20 x 1
     genes
0     XRS2
1     SKI2
2  YGL214W
3     SLX8
4     TOP3
Empty DataFrame
Columns: [genes, orf]
Index: []
(20, 1)
Original data dimensions: 1 x 1
  genes
0  PKC1
Empty DataFrame
Columns: [genes, orf]
Index: []
(1, 1)
Original data dimensions: 10 x 1
     genes
0     SRS2
1     SKI2
2     SKI7
3    RAD51
4  YDL073W
Empty DataFrame
Columns: [genes, orf]
Index: []
(10, 1)
Original data dimensions: 9 x 1
   genes
0   PKC1
1   TRE2
2   ARP9
3  PRP43
4   PRE5
Empty DataFrame
Columns: [genes, orf]
Index: []
(9, 1)


In [8]:
original_data = pd.concat(original_data_list, axis=1)

In [17]:
# Assuming (a bit reluctantly) that all genes were tested in all the screens
original_data[original_data.isnull()] = 0

In [24]:
original_data.index.rename('orf', inplace=True)

# Prepare the final dataset

In [25]:
data = original_data.copy()

In [26]:
dataset_ids = [16634, 16637, 16635, 16636]
datasets = datasets.reindex(index=dataset_ids)

In [27]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [28]:
data.head()

dataset_id,16634,16637,16635,16636
data_type,value,value,value,value
orf,,,,
YBR098W,-1.0,0.0,0.0,0.0
YDR266C,-1.0,0.0,0.0,0.0
YDR369C,-1.0,0.0,0.0,0.0
YER116C,-1.0,0.0,0.0,0.0
YFR019W,-1.0,0.0,0.0,0.0


## Subset to the genes currently in SGD

In [29]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [30]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16634,16637,16635,16636
,data_type,value,value,value,value
gene_id,orf,,,,
293,YBR098W,-1.0,0.0,0.0,0.0
2393,YDR266C,-1.0,0.0,0.0,0.0
2496,YDR369C,-1.0,0.0,0.0,0.0
845,YER116C,-1.0,0.0,0.0,0.0
1811,YFR019W,-1.0,0.0,0.0,0.0


# Normalize

In [31]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [32]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [33]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16634,16637,16635,16636,16634,16637,16635,16636
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
293,YBR098W,-1.0,0.0,0.0,0.0,-15.10629,0.0,0.0,0.0
2393,YDR266C,-1.0,0.0,0.0,0.0,-15.10629,0.0,0.0,0.0
2496,YDR369C,-1.0,0.0,0.0,0.0,-15.10629,0.0,0.0,0.0
845,YER116C,-1.0,0.0,0.0,0.0,-15.10629,0.0,0.0,0.0
1811,YFR019W,-1.0,0.0,0.0,0.0,-15.10629,0.0,0.0,0.0


# Print out

In [34]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [35]:
from IO.save_data_to_db3 import *

In [37]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/4 [00:00<?, ?it/s]

Deleting all datasets for PMID 21723647...
Inserting the new data...


100%|██████████| 4/4 [00:00<00:00,  7.58it/s]

Updating the data_modified_on field...
